In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp pipeline.core

In [ ]:
# hide

from kaggle_m5_nbdev.core import test_eq, test_err, configure_logging
log = configure_logging('./tmp', 'test_log', con_log_lvl='DEBUG', date_format='XXXX-XX-XX XX:XX:XX')

In [ ]:
#export

from kaggle_m5_nbdev.core import read_series_sample, melt_sales_series, extract_day_ids, join_w_calendar, join_w_prices
from kaggle_m5_nbdev.core import to_parquet, get_submission_template_melt
from kaggle_m5_nbdev.petastorm import ParquetIterableDataset
import os

def prepare_data_on_disk(log, n_sample_series, processed_dir, raw_dir, force_data_prep):
    expected_path = f'{processed_dir}/sales_series_melt.parquet'
    if os.path.exists(expected_path) and not force_data_prep:
        log.info(f'Found parquet file ({expected_path})- skipping the prep')
        return

    log.info(f'Not found parquet file ({expected_path}) - preparing the data')

    sales_series = read_series_sample(log, n_sample_series)
    sales_series = melt_sales_series(sales_series)
    sales_series = extract_day_ids(sales_series)
    sales_series = join_w_calendar(sales_series, raw_dir)
    sales_series = join_w_prices(sales_series, raw_dir).persist()
    to_parquet(sales_series, 'sales_series_melt.parquet', processed_dir, log)

In [ ]:
prepare_data_on_disk(log, n_sample_series=10, processed_dir='./tmp', raw_dir='raw', force_data_prep=True)
prepare_data_on_disk(log, n_sample_series=10, processed_dir='./tmp', raw_dir='raw', force_data_prep=False)

 INFO [XXXX-XX-XX XX:XX:XX] root:  Not found parquet file (./tmp/sales_series_melt.parquet) - preparing the data
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Read 10 series
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Setting index
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Setting index - done
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Dropping 'day_date' from Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day_id',
       'sales', 'day_date', 'day_date_str', 'month_id', 'date', 'wm_yr_wk',
       'snap_flag', 'sell_price', 'sales_dollars'],
      dtype='object')
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding: id
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding: item_id
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding: dept_id
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding: cat_id
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding: store_id
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding: state_id
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding: day_date_str
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding: date
DEBUG [XX

In [ ]:
# TODO: move to core (don't forget to add import here)

from sklearn.preprocessing import LabelEncoder
import dask.dataframe as dd
import numpy as np

def load_encoders(processed):
    def _load(fn):
        l = LabelEncoder()
        l.classes_ = np.load(f'{processed}/{fn}', allow_pickle=True)
        return l

    encoders_paths = filter(lambda p: p.endswith('.npy'), os.listdir(processed))
    encoders = {fn[:-len('.npy')]:_load(fn) for fn in encoders_paths}

    return encoders

def encode(log, me, processed):
    encoders = load_encoders(processed)
    continuous_cols = ['sell_price']

    for col in me.columns:
        dtype_str = str(me[col].dtype)
        if col in continuous_cols:
            log.debug(f"Encoding {col} ({dtype_str}) as float32 just in case for pytorch")
            me[col] = me[col].astype('float32')
            continue

        log.debug(f"Encoding {col} ({dtype_str}) as categorical ")

        unlabelable = ~me[col].isin(encoders[col].classes_)
        unlabelable_count = unlabelable.sum()
        if unlabelable_count > 0:
            default_label = encoders[col].classes_[0]
            log.warning(f"{unlabelable_count} entries for {col} can't be labeled. Defaulting to {default_label} e.g.\n {me[unlabelable][col][:3].values}")
            me.loc[unlabelable, col] = default_label

        me[col] = encoders[col].transform(me[col])

    return me

In [ ]:
raw = 'raw'
processed = './tmp'

In [ ]:
def prepare_test_data_on_disk(log, raw, processed, force_data_prep):
    expected_path = f'{processed}/test_series_melt.parquet'
    if os.path.exists(expected_path) and not force_data_prep:
        log.info(f'Found parquet file ({expected_path})- skipping the prep')
        return

    template = get_submission_template_melt(raw)
    test_data = encode(log, template, processed)
    to_parquet(test_data, 'test_series_melt.parquet', processed, log)

In [ ]:
prepare_test_data_on_disk(log, raw='raw', processed='./tmp', force_data_prep=True)
prepare_test_data_on_disk(log, raw='raw', processed='./tmp', force_data_prep=False)

DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding id (object) as categorical 
WARNING [XXXX-XX-XX XX:XX:XX] root:   1707160 entries for id can't be labeled. Defaulting to FOODS_1_152_WI_3_validation e.g.
 ['HOBBIES_1_001_CA_1_validation' 'HOBBIES_1_002_CA_1_validation'
 'HOBBIES_1_003_CA_1_validation']
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding date (object) as categorical 
WARNING [XXXX-XX-XX XX:XX:XX] root:   1707440 entries for date can't be labeled. Defaulting to 2011-01-29 e.g.
 [datetime.date(2016, 6, 20) datetime.date(2016, 6, 20)
 datetime.date(2016, 6, 20)]
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding sell_price (float64) as float32 just in case for pytorch
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding cat_id (object) as categorical 
DEBUG [XXXX-XX-XX XX:XX:XX] root:   Encoding dept_id (object) as categorical 
WARNING [XXXX-XX-XX XX:XX:XX] root:   455840 entries for dept_id can't be labeled. Defaulting to FOODS_1 e.g.
 ['HOBBIES_1' 'HOBBIES_1' 'HOBBIES_1']
DEBUG [XXXX-XX-XX XX:XX:

In [ ]:
from torch.utils.data import TensorDataset, DataLoader as TorchDataLoader, IterableDataset
from collections import OrderedDict
def setup_data_loaders(processed, log):
    batch = 128

    train_ds = ParquetIterableDataset(f'file:{processed}/sales_series_melt.parquet', log, '.*part.(?!1).*')
    valid_ds = ParquetIterableDataset(f'file:{processed}/sales_series_melt.parquet', log, '.*part.1.*')
    test_ds  = ParquetIterableDataset(f'file:{processed}/test_series_melt.parquet', log)

    train_dl = TorchDataLoader(train_ds, batch_size=batch, shuffle=False, num_workers=0, drop_last=False)
    valid_dl = TorchDataLoader(valid_ds, batch_size=batch, shuffle=False, num_workers=0, drop_last=False)
    test_dl  = TorchDataLoader(test_ds,  batch_size=batch, shuffle=False, num_workers=0, drop_last=False)

    data = OrderedDict()
    data["train"] = train_dl
    data["valid"] = valid_dl
    data["test"]  = test_dl

    return data

In [ ]:
dls = setup_data_loaders(processed=processed, log=log)

DEBUG [XXXX-XX-XX XX:XX:XX] root:    file:./tmp/sales_series_melt.parquet -> 1 files out of 4 remained after applying filter (.*part.(?!1).*)
DEBUG [XXXX-XX-XX XX:XX:XX] root:    file:./tmp/sales_series_melt.parquet -> 1 files out of 4 remained after applying filter (.*part.1.*)


In [ ]:
def log_batch(model, data, log, log_stage):
   for key, dl in data.items():
        batch = next(iter(dl))
        log.debug(f"{key} model out @ {log_stage} {model.forward(batch['features']).transpose(1, 0)}")

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

from catalyst.dl import SupervisedRunner
from catalyst.utils import set_global_seed

class Net(nn.Sequential):
    def __init__(self, num_features):
        layers = []
        layer_dims = [num_features, 200,200,20,20,1]
        for in_features, out_features in zip(layer_dims[:-1], layer_dims[1:]):
            l = nn.Linear(in_features, out_features)
            # Note to self: loss @ init is quite important!
            torch.nn.init.xavier_uniform_(l.weight) 
            torch.nn.init.zeros_(l.bias)

            layers.append(l)
            layers.append(nn.ReLU())
        super(Net, self).__init__(*layers)

class MyLoss(nn.MSELoss):
    def __init__(self):
        super(MyLoss, self).__init__()

    def forward(self, inp, target):
        return super().forward(inp, target)

def do_train(data, log, log_dir):
    model = Net(num_features = 2)
    runner = SupervisedRunner()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    criterion = MyLoss()

    log_batch(model, data, log, "init")

    log.debug("Starting training")
    runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        loaders=data,
        logdir=f"{log_dir}/run",
        load_best_on_end=True,
        num_epochs=1)

    log_batch(model, data, log, "exit")

/usr/local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [ ]:
do_train(dls, log, log_dir='./tmp')

DEBUG [XXXX-XX-XX XX:XX:XX] root:       Iterator created on file:./tmp/sales_series_melt.parquet
/usr/local/lib/python3.7/site-packages/petastorm/compat.py:39: ResourceWarning:

unclosed file <_io.BufferedReader name='./tmp/sales_series_melt.parquet/part.0.parquet'>

/usr/local/lib/python3.7/site-packages/petastorm/workers_pool/thread_pool.py:62: ResourceWarning:

unclosed file <_io.BufferedReader name='./tmp/sales_series_melt.parquet/part.0.parquet'>

/usr/local/lib/python3.7/site-packages/petastorm/arrow_reader_worker.py:53: FutureWarning:

Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute

/usr/local/lib/python3.7/site-packages/petastorm/workers_pool/thread_pool.py:187: PendingDeprecationWarning:

isAlive() is deprecated, use is_alive() instead

DEBUG [XXXX-XX-XX XX:XX:XX] root:   train model out @ init tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0